In [1]:
import torch
print("MPS Available:", torch.backends.mps.is_available())

MPS Available: True


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd

# ✅ Move computations to GPU if available
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
print(f"Using device: {device}")

# Load Titanic dataset using Pandas
df = pd.read_csv("titanic.csv")

# Preprocessing: Fill missing values, encode categorical features
df["Age"].fillna(df["Age"].median(), inplace=True)
df["Fare"].fillna(df["Fare"].median(), inplace=True)
df["Sex"] = df["Sex"].map({"male": 0, "female": 1})

# Normalize numerical columns
for col in ["Age", "Fare"]:
    df[col] = (df[col] - df[col].min()) / (df[col].max() - df[col].min())

# Select features and target
features = ["Pclass", "Sex", "Age", "SibSp", "Parch", "Fare"]
label = "Survived"

# Convert to PyTorch tensors and move to GPU
X = torch.tensor(df[features].values, dtype=torch.float32).to(device)
y = torch.tensor(df[label].values, dtype=torch.float32).view(-1, 1).to(device)

# Define the neural network
class TitanicNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(TitanicNN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.sigmoid(x)
        return x

# Initialize the model and move to GPU
model = TitanicNN(input_size=len(features), hidden_size=15, output_size=1).to(device)

# Loss function and optimizer
criterion = nn.BCELoss()  # Binary Cross Entropy Loss for classification
optimizer = optim.Adam(model.parameters(), lr=0.01)

# Training loop
epochs = 1000
for epoch in range(epochs):
    optimizer.zero_grad()
    outputs = model(X)
    loss = criterion(outputs, y)
    loss.backward()
    optimizer.step()

    if epoch % 100 == 0:
        print(f"Epoch {epoch}, Loss: {loss.item()}")

# Evaluate accuracy
with torch.no_grad():
    predictions = model(X)
    predictions = (predictions > 0.5).float()
    accuracy = (predictions == y).sum().item() / len(y)
    print(f"✅ Accuracy: {accuracy * 100:.2f}%")

Using device: mps
Epoch 0, Loss: 0.6910756826400757
Epoch 100, Loss: 0.39791983366012573
Epoch 200, Loss: 0.3723345100879669
Epoch 300, Loss: 0.35688722133636475
Epoch 400, Loss: 0.34792259335517883
Epoch 500, Loss: 0.34193429350852966
Epoch 600, Loss: 0.33757540583610535
Epoch 700, Loss: 0.33397436141967773
Epoch 800, Loss: 0.33098170161247253
Epoch 900, Loss: 0.3282948136329651
✅ Accuracy: 85.86%
